In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

In [12]:
df = spark.read.csv("session_duration.csv", header=True)

In [13]:
df.toPandas()

,userId,sessionId,timestamp
0,A,1,1523849238491
1,A,1,1523849238492
2,A,1,1523849238493
3,A,1,1523849238496
4,A,2,1523849238492
5,A,2,1523849238495


In [15]:
df_with_session_length = df.groupBy(['userId', 'sessionId'])\
    .agg(F.min('timestamp').alias('session_start'), F.max('timestamp').alias('session_end'))\
    .withColumn('feature_session_duration', F.col('session_end') - F.col('session_start'))\
    .select('userId', 'sessionId', 'feature_session_duration')
df_with_session_length.toPandas()

,userId,sessionId,feature_session_duration
0,A,1,5.0
1,A,2,3.0


In [48]:

dfc2 = spark.read.csv("session_duration.csv", header=True)

In [49]:
dfc2.toPandas()

,sessionId,sessionId:,timestamp
0,A,1,1523849238491
1,None,1,1523849238492
2,A,None,1523849238493
3,A,1,1523849238496
4,A,2,1523849238492
5,A,2,1523849238495


In [50]:
columns_to_merge = {}
for col_name in dfc2.columns:
    new_col_name = col_name.replace(".","_")
    dfc2 = dfc2.withColumnRenamed(col_name, new_col_name)
    if ":" in new_col_name:
        error_column_name = new_col_name.replace(":","_exception")
        columns_to_merge[new_col_name.replace(":","")] = error_column_name
        dfc2 = dfc2.withColumnRenamed(new_col_name, error_column_name)
        


In [51]:
dfc2.toPandas()

,sessionId,sessionId_exception,timestamp
0,A,1,1523849238491
1,None,1,1523849238492
2,A,None,1523849238493
3,A,1,1523849238496
4,A,2,1523849238492
5,A,2,1523849238495


In [52]:
for valid_column_name, error_column_name in columns_to_merge.items():
    print(valid_column_name)
    print(error_column_name)
    if (valid_column_name in dfc2.columns) & (error_column_name in dfc2.columns):
        print("i")
        dfc2 = dfc2.withColumn(valid_column_name, F.coalesce(F.col(error_column_name), F.col(valid_column_name)))
        dfc2 = dfc2.drop(error_column_name)

sessionId
sessionId_exception
i


In [53]:
dfc2.toPandas()

,sessionId,timestamp
0,1,1523849238491
1,1,1523849238492
2,A,1523849238493
3,1,1523849238496
4,2,1523849238492
5,2,1523849238495
